In [21]:
from hashlib import sha1
import hmac
import requests
import json
import os
import binascii
from urllib import parse
import html2text as html2text
import requests
import json
import pandas as pd

In [ ]:
def getUrl(request):
    ptv_key = os.getenv('ptv_key')
    devId = 3002228
    request = request + ('&' if ('?' in request) else '?')
    raw = request+'devid={0}'.format(devId)
    #print(raw.encode())
    key_bin = ptv_key.encode()
    raw_bin = raw.encode() 
    hashed = hmac.new(key_bin, raw_bin, sha1)
    signature = hashed.hexdigest()
    return 'http://timetableapi.ptv.vic.gov.au'+raw+'&signature={1}'.format(devId, signature)
#print(getUrl('/v3/route_types'))



def download_get(url_name,file_path):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    with open(file_path,'w') as f:
        f.write(response.text)
def download_get_to_dict(url_name):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    return json.loads(response.text)


routes_json_file = '../data/ptv_data/routes.json'
with open(routes_json_file) as f:
    route_dict = json.load(f)
    # for different transportation store the route infomration
    stops_files = [open(f'../data/ptv_data/stops_file_{i}.json','a')  for i in range(5)]
    for stop_file in stops_files:
        stop_file.write('[\n')
    is_first_line = [True,True,True,True,True]

    for route_item in route_dict['routes']:
        route_id = route_item['route_id']
        route_type = route_item['route_type']
        url_name = f'/v3/stops/route/{route_id}/route_type/{route_type}'
        res = download_get_to_dict(url_name)
        stops = res['stops']
        for stop in stops:
            if not is_first_line[route_type]:
                stops_files[route_type].write(',\n')
            json.dump(stop,stops_files[route_type])
            is_first_line[route_type] = False
    
    for stop_file in stops_files:
        stop_file.write('\n]')
        stop_file.close()  

In [16]:
def to_lower(text):
    text = text.lower()
    return text

In [17]:
# train stops
# import json
# with open ('../data/ptv_data/stops_file_0.json') as fp:
#     data = json.load(fp)
train_stops_file = pd.read_json('../data/ptv_data/stops_file_0.json')
train_stops_file = train_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(to_lower)
train_stops_file['stop_name'] = train_stops_file['stop_name'].drop_duplicates()
train_stops_file = train_stops_file.dropna()
train_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ashburton,-37.868317,145.079666,1002,Alamein Station
1,ashburton,-37.861970,145.081345,1010,Ashburton Station
2,hawthorn east,-37.822400,145.045837,1012,Auburn Station
3,burnley,-37.827557,145.007553,1030,Burnley Station
4,glen iris,-37.851562,145.080500,1031,Burwood Station
...,...,...,...,...,...
347,williams landing,-37.869870,144.747452,1225,Williams Landing Station
348,yarraville,-37.815850,144.889938,1216,Yarraville Station
355,williamstown,-37.857330,144.889069,1146,North Williamstown Station
361,williamstown,-37.867756,144.905319,1211,Williamstown Station


In [18]:
def change_name(text):
    if(text == 'melbourne city'):
        return 'melbourne'
    else:
        return text
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(change_name)

In [ ]:
# tram stops
tram_stops_file = pd.read_json('../data/stops_file_1.json')
tram_stops_file = tram_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(to_lower)
tram_stops_file['stop_name'] = tram_stops_file['stop_name'].drop_duplicates()
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(change_name)
tram_stops_file = tram_stops_file.dropna()
tram_stops_file

In [365]:
# bus stops
bus_stops_file = pd.read_json('../data/stops_file_2.json')
bus_stops_file = bus_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(to_lower)
bus_stops_file['stop_name'] = bus_stops_file['stop_name'].drop_duplicates()
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(change_name)
bus_stops_file = bus_stops_file.dropna()
bus_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,avondale heights,-37.762234,144.859680,23570,Avondale Heights Reserve/Doyle St
1,maribyrnong,-37.769444,144.872772,31671,Canning Reserve/Canning St
3,avondale heights,-37.769413,144.857254,25271,Canning St/Herbert St
4,maribyrnong,-37.769013,144.877823,11239,Central Park Ave/Cordite Ave
5,avondale heights,-37.759260,144.857391,25277,Davis Ave/Skewes St
...,...,...,...,...,...
29797,hampton park,-38.030907,145.258270,19099,Robert Booth Reserve/Somerville Rd
29798,hampton park,-38.042637,145.270248,17513,Robert Ct/Huntington Dr
29799,narre warren south,-38.050587,145.296158,17528,Seebeck Dr/Sneddon Dr
29802,hampton park,-38.039837,145.280609,17583,The Ridge/Ralph Cres


In [366]:
# vline stops
vline_stops_file = pd.read_json('../data/stops_file_3.json')
vline_stops_file = vline_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(to_lower)
vline_stops_file['stop_name'] = vline_stops_file['stop_name'].drop_duplicates()
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(change_name)
vline_stops_file = vline_stops_file.dropna()
vline_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy
...,...,...,...,...,...
1381,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St
1383,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy
1391,clunes,-37.303142,143.781570,1591,Clunes Railway Station
1405,talbot,-37.172900,143.705627,1594,Talbot Railway Station


In [ ]:

url = "https://vahi.vic.gov.au/views/ajax?_wrapper_format=drupal_ajax"
for i in range(1):
    post_body = parse.urlencode({"view_name":"hospital_and_health_services","view_display_id":"page_1","page":i})
    response = requests.post(url= url,headers={
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "zh-CN,zh;q=0.9",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-ch-ua": "\"Chromium\";v=\"104\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"104\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"
      },data = post_body)
    print(response.status_code)
    print(response.text)
    with open(f'../data/raw/v2_ajax_download_{i:02d}','w') as f:
        f.write(response.text)



with open('../data/raw/v2_ajax_download_00') as f:
    d = json.load(f)
    print(len(d))
    geofield_google_map = d[0]["settings"]["geofield_google_map"]
    with open('../data/raw/v2_ajax_download.csv','w') as csv_file:
        csv_file.write('"name","address"\n')
        for geofield in geofield_google_map:
            for feature in geofield_google_map[geofield]["data"]["features"]:
                #print(feature)
                name_with_tag = feature["properties"]["data"]["view_node"]
                name = name_with_tag[name_with_tag.index('>')+1 : name_with_tag.index('<',1)]
                address = feature["properties"]["data"]["field_address"]
                #print(html2text.html2text(),end=";")
                csv_file.write(f'"{name}","{address}"\n')

In [6]:
hospital = pd.read_csv("../data/raw/v2_ajax_download.csv")

In [7]:
import re
for i in range(len(hospital)):
    postcode_text = re.findall(r'VIC [0-9]+',hospital['address'][i])
    if len(postcode_text) == 1:
        hospital.loc[i,'postcode'] = postcode_text[0].split(' ')[1]
    elif len(postcode_text) == 0:
        hospital.loc[i,'postcode'] = None
    else:
        hospital.loc[i,'postcode'] = postcode_text[0].split(' ')[1]

In [521]:
# store hospital in VIC
hospital = hospital.dropna().reset_index()
hospital = hospital.drop('index',axis = 1)
hospital.to_csv('../data/hospital_csv')

In [9]:
def to_lower(text):
    return text.lower()
geo_df = pd.read_excel("../data/postcode_in_VIC.csv")
geo_df = geo_df[geo_df['Postcode'] > 0 ]
geo_df = geo_df[geo_df['Postcode'] < 8000 ]
geo_df['Locality'] = geo_df['Locality'].apply(to_lower)

In [11]:
import math
EARTH_REDIUS = 6378.137


def rad(d):
    return d * math.pi / 180.0


def getDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2), 2) + math.cos(radLat1) * math.cos(radLat2) * math.pow(math.sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s


def find_postcode(geo_df,lat, lng, suburb):
    if(suburb == 'melbourne'):
        distance = 10000
        result = None
        for index, item in geo_df.iterrows():
            postcode = item['Postcode']
            lat2, lng2 = item['Latitude'], item['Longitude']
            d = getDistance(lat, lng, lat2, lng2)
            if (d < distance):
                distance = d
                result = postcode
        return int(result)
    else:
        for index, item in geo_df.iterrows():
            locality = item['Locality']
            if locality == suburb:
                return int(item['Postcode'])


In [ ]:
train_stops_file['postcode'] = None
for index, item in train_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    train_stops_file.loc[index, 'postcode'] = postcode
train_stops_file = train_stops_file.reset_index().drop('index', axis = 1)

In [372]:
tram_stops_file['postcode'] = None
for index, item in tram_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    tram_stops_file.loc[index, 'postcode'] = postcode
tram_stops_file = tram_stops_file.reset_index().drop('index', axis = 1)

In [373]:
bus_stops_file['postcode'] = None
for index, item in bus_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    bus_stops_file.loc[index, 'postcode'] = postcode
bus_stops_file = bus_stops_file.reset_index().drop('index', axis = 1)

In [374]:
vline_stops_file['postcode'] = None
for index, item in vline_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    vline_stops_file.loc[index, 'postcode'] = postcode
vline_stops_file = vline_stops_file.reset_index().drop('index', axis = 1)

In [539]:
train_stops_file.to_csv('../data/train_stops_file.csv')
tram_stops_file.to_csv('../data/tram_stops_file.csv')
bus_stops_file.to_csv('../data/bus_stops_file.csv')
vline_stops_file.to_csv('../data/vline_stops_file.csv')